In [ ]:
%%capture
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Developer/trading/cnn_rl/src
#%cd /content/drive/MyDrive/trading/cnn_rl/src
   
%pip install pyts

In [ ]:
from helper_config import Config
from helper_env import PortfolioEnvironment
from helper_agent import DQLAgent
from helper_dataset import RLDataset
cfg = Config()
cfg.data_dir = '../data'
cfg.models_dir = '../models/rl'
cfg.base_model_path = '../models/1dcnn_gadf_zero_centered_ohlc/baseline/model.pth'
target_assets = len(cfg.tickers)

### RL / POLICY NETWORK CONFIGURATION
cfg.input_dim       = len(cfg.tickers) * (4)
cfg.output_dim      = len(cfg.tickers)  # values between 0-1 for each asset, potraying funds to be invested
cfg.learning_rate   = 1e-3
cfg.gamma           = 0.99
cfg.epsilon         = 1.0
cfg.epsilon_decay   = 0.9995
cfg.epsilon_min     = 0.01
cfg.reply_capacity  = 10000
cfg.batch_size      = 32
cfg.hidden_size     = 128
cfg.riskfree_rate   = 0.02
cfg.initial_balance = 1_000_000

prep = RLDataset(cfg)  # we create dataset for the RL model
df_norm_close, df_prices, images, labels = prep.prepare(kind = 'train') # here we import the data

# define environment
env = PortfolioEnvironment(cfg, df_norm_close, df_prices, images, labels, window_size = 2000) # this class defines the trading env, norm_close + logits from CNN goes to Policy Network

# define agent
agent = DQLAgent(cfg, name = 'baseline')
results = agent.train(env, episodes = 20000) # 